# Integración y Aplicación Práctica de la Ingeniería de Características

En este capítulo, integraremos las técnicas y conceptos de ingeniería de características previamente discutidos, aplicándolos en un contexto práctico para resolver un problema real. Esta sección combinará la transformación, creación y selección de características, junto con la implementación de funciones personalizadas y la manipulación avanzada de datos.


**Preparación de los Datos**

La preparación de los datos es un paso fundamental en cualquier proyecto de machine learning. Involucra la limpieza, transformación y selección de las características más relevantes para mejorar el rendimiento del modelo. A continuación, se describe un flujo de trabajo típico para la preparación de datos en un entorno de machine learning.


**Limpieza y Transformación de Datos**

La limpieza de datos es esencial para eliminar o corregir valores faltantes, duplicados o incorrectos. La transformación de datos incluye la normalización, estandarización y aplicación de técnicas avanzadas de ingeniería de características, como las vistas en los capítulos anteriores.

In [5]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Cargar un conjunto de datos de ejemplo
data = pd.read_csv('../../data/dataset.csv')

# Limpiar datos: eliminar duplicados y manejar valores faltantes
data.drop_duplicates(inplace=True)
data.fillna(data.mean(), inplace=True)

# Separar la variable objetivo del resto de las características
target = data['target']  # Extraemos la columna 'target'
features = data.drop(columns=['target'])  # Excluimos 'target' de las características

# Normalizar características numéricas
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features.select_dtypes(include=[np.number]))

# Crear características polinómicas
poly = PolynomialFeatures(degree=2, include_bias=False)
features_poly = poly.fit_transform(features_scaled)

# Agregar las características polinómicas al DataFrame original
features_final = pd.DataFrame(features_poly, columns=poly.get_feature_names_out(features.columns))

# Agregar nuevamente la columna 'target'
data_final = pd.concat([features_final, target.reset_index(drop=True)], axis=1)

print(data_final.head())


   feature1  feature2  feature3  feature4  feature1^2  feature1 feature2  \
0 -0.323112 -1.599230  0.426196 -1.505121    0.104401           0.516731   
1  1.623434  0.475167 -1.484683  0.137700    2.635538           0.771402   
2  0.884509 -0.629113 -1.219270  0.169484    0.782357          -0.556456   
3  0.434049  0.036822  1.304832  0.500994    0.188399           0.015983   
4 -1.061365  1.404924  0.304251  0.804649    1.126495          -1.491136   

   feature1 feature3  feature1 feature4  feature2^2  feature2 feature3  \
0          -0.137709           0.486323    2.557535          -0.681586   
1          -2.410285           0.223546    0.225784          -0.705473   
2          -1.078456           0.149910    0.395783           0.767059   
3           0.566361           0.217456    0.001356           0.048047   
4          -0.322921          -0.854026    1.973810           0.427449   

   feature2 feature4  feature3^2  feature3 feature4  feature4^2  target  
0           2.407034    

**Creación y Selección de Características**

La creación de nuevas características y la selección de las más relevantes es clave para optimizar el rendimiento del modelo. Usando técnicas avanzadas como operadores sobrecargados y funciones personalizadas, se pueden generar características que capturen relaciones complejas entre los datos.



In [6]:
# Crear nuevas características utilizando operadores sobrecargados
data_final['Ratio_Feature1_Feature2'] = data_final['feature1'] / data_final['feature2']
data_final['Product_Feature3_Feature4'] = data_final['feature3'] * data_final['feature4']

# Seleccionar las características más importantes utilizando un modelo de bosque aleatorio
X = data_final.drop('target', axis=1)
y = data_final['target']

model = RandomForestClassifier(random_state=42)
model.fit(X, y)

importances = model.feature_importances_
important_features = X.columns[importances > np.percentile(importances, 75)]

print(f"Características seleccionadas: {important_features}")

Características seleccionadas: Index(['feature2', 'feature4', 'feature2^2', 'feature2 feature4'], dtype='object')


**Aplicación de Funciones Personalizadas**

Las funciones personalizadas permiten transformar los datos de manera específica para el dominio del problema. Estas funciones pueden escalar, normalizar o transformar las características de acuerdo con las necesidades particulares del proyecto

In [7]:
# Definir una función personalizada para escalar datos entre 0 y 1
def escalar_personalizado(x, min_val, max_val):
    return (x - min_val) / (max_val - min_val)

# Aplicar la función personalizada a una característica específica
data_final['Feature_Scaled'] = data_final['feature1'].apply(escalar_personalizado, args=(data_final['feature1'].min(), data_final['feature1'].max()))

print(data_final['Feature_Scaled'].head())

0    0.376025
1    0.963140
2    0.740267
3    0.604399
4    0.153354
Name: Feature_Scaled, dtype: float64


**Entrenamiento y Evaluación del Modelo**

Finalmente, el conjunto de datos procesado se utiliza para entrenar un modelo de machine learning. La evaluación del modelo se realiza utilizando métricas como la precisión, la sensibilidad o el error cuadrático medio, dependiendo del tipo de problema.

In [8]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X[important_features], y, test_size=0.3, random_state=42)

# Entrenar un modelo de Random Forest
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Evaluar el modelo
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.4f}")

Precisión del modelo: 0.5000
